In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
import string

In [ ]:
dfa = pd.read_csv('DBLP-ACM/ACM.csv')
dfa = dfa.set_index('id', drop=True)
dfa.loc[:, 'title'] = dfa.title.str.lower()\
    .str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))\
    .map(lambda x: ' '.join(x.split()))
dfa.loc[dfa.authors.notna(), 'authors'] = dfa.loc[dfa.authors.notna(), 'authors']\
    .map(lambda x: ', '.join(sorted(x.split(', '))))
dfa

In [ ]:
dfb = pd.read_csv('DBLP-ACM/DBLP2.csv', encoding='latin_1')
dfb = dfb.set_index('id', drop=True)
dfb.loc[:, 'title'] = dfb.title.str.lower()\
    .str.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))\
    .map(lambda x: ' '.join(x.split()))
dfb.loc[dfb.authors.notna(), 'authors'] = dfb.loc[dfb.authors.notna(), 'authors']\
    .map(lambda x: ', '.join(sorted(x.split(', '))))
dfb

In [ ]:
matches = []
for year in dfa.year.unique():
    dfa_block = dfa.loc[dfa.year == year, 'title']
    dfb_block = dfb.loc[dfb.year == year, 'title']
    for index, title_a in dfa_block.items():
        for match in dfb_block.index[dfb_block == title_a].tolist():
            matches.append([match, index])

In [ ]:
df = pd.DataFrame(matches, columns=['idDBLP', 'idACM'])
df['pred'] = 1
df

In [ ]:
pm = pd.read_csv('DBLP-ACM/DBLP-ACM_perfectMapping.csv')
pm['true'] = 1
pm

In [ ]:
full = pd.merge(df, pm, how='outer').fillna(0)
full

In [ ]:
pr = precision_score(full.true, full.pred)
re = recall_score(full.true, full.pred)
f1 = f1_score(full.true, full.pred)
pr, re, f1